In [1]:
import ee
import datetime
import pandas as pd
import geopandas as gpd
import numpy as np
import geemap
from shapely.geometry import polygon
import re

# Initialize the Earth Engine module.
ee.Initialize()
pd.set_option('display.max_rows', None)


In [3]:
# read in data
fires = pd.read_csv("../data/outputs/on/on_defoliation_history_gt90_final.csv")
# get fire names
fire_names = fires['Fire_ID'].tolist()

# bring in shapefile of all quebec fires
data = gpd.read_file("../data/inputs/on/fire/ON_FirePerimeters_85to2020_v00.shp")


# filter data by fire names
data = data[data['Fire_ID'].isin(fire_names)]


# if columns are Fire_Year and defoliated ajust them to match the function
data['Fire_Year'] = data['Fire_Year'].astype(float)
data['Fire_ID'] = data['Fire_ID'].astype(str)

# if there is no index, create index
data['id'] = data.index
data.info()




<class 'geopandas.geodataframe.GeoDataFrame'>
Index: 807 entries, 0 to 1361
Data columns (total 26 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   OGF_ID      807 non-null    int64   
 1   r_id        807 non-null    object  
 2   raster_id   807 non-null    object  
 3   Fire_ID     807 non-null    object  
 4   FIRE_DISTU  807 non-null    object  
 5   FIRE_TYPE_  807 non-null    object  
 6   Fire_Year   807 non-null    float64 
 7   FIRE_GENER  807 non-null    object  
 8   FIRE_RESPO  807 non-null    object  
 9   FIRE_START  807 non-null    object  
 10  FIRE_OUT_D  807 non-null    object  
 11  FIRE_FINAL  807 non-null    int64   
 12  LOCATION_A  807 non-null    object  
 13  BUSINESS_E  738 non-null    object  
 14  GENERAL_CO  426 non-null    object  
 15  GEOMETRY_U  70 non-null     object  
 16  EFFECTIVE_  807 non-null    object  
 17  SYSTEM_DAT  807 non-null    object  
 18  OBJECTID    807 non-null    int64   
 19  SHAP

Test functions: create test_data for 10 fires


In [35]:
data_10 = data.sample(n=30, random_state=1)
data_10

,OGF_ID,r_id,raster_id,Fire_ID,FIRE_DISTU,FIRE_TYPE_,Fire_Year,FIRE_GENER,FIRE_RESPO,FIRE_START,...,EFFECTIVE_,SYSTEM_DAT,OBJECTID,SHAPEAREA,SHAPELEN,old_f_id,RBR,TIF,geometry,id
14,46002052,RED1_2002_15_rbr_.tif,RED1_2002_15_rbr,RED1_2002_15,RED1,OFR,2002.0,LTG,MON,2002-07-16,...,2003-07-30,2005-04-28,20,0,0,15,rbr,.tif,"POLYGON ((-92.54654 54.29048, -92.54673 54.290...",14
484,46001374,GER20_1990_740_rbr_.tif,GER20_1990_740_rbr,GER20_1990_740,GER20,IFR,1990.0,LTG,MON,1990-07-07,...,2003-07-31,2005-04-28,852,0,0,740,rbr,.tif,"POLYGON ((-86.38440 52.36671, -86.38762 52.366...",484
183,46001696,NIP60_1996_227_rbr_.tif,NIP60_1996_227_rbr,NIP60_1996_227,NIP60,IFR,1996.0,LTG,SUP,1996-06-12,...,2003-07-31,2005-04-28,284,0,0,227,rbr,.tif,"POLYGON ((-86.65806 50.64998, -86.65783 50.649...",183
600,46001976,COC22_1999_954_rbr_.tif,COC22_1999_954_rbr,COC22_1999_954,COC22,OFR,1999.0,LTG,MON,1999-08-29,...,2003-07-31,2005-04-28,1089,0,0,954,rbr,.tif,"POLYGON ((-89.12290 54.11994, -89.12258 54.120...",600
791,46001586,NIP66_1995_1281_rbr_.tif,NIP66_1995_1281_rbr,NIP66_1995_1281,NIP66,IFR,1995.0,LTG,SUP,1995-06-19,...,2003-07-31,2005-04-28,1447,0,0,1281,rbr,.tif,"MULTIPOLYGON (((-88.02941 51.09015, -88.02976 ...",791
608,46001601,RED81_1995_964_rbr_.tif,RED81_1995_964_rbr,RED81_1995_964,RED81,IFR,1995.0,LTG,SUP,1995-06-15,...,2003-07-31,2005-04-28,1100,0,0,964,rbr,.tif,"MULTIPOLYGON (((-92.82662 50.80599, -92.82742 ...",608
900,46001635,SLK69_1995_1451_rbr_.tif,SLK69_1995_1451_rbr,SLK69_1995_1451,SLK69,IFR,1995.0,LTG,SUP,1995-06-15,...,2003-07-31,2005-04-28,1638,0,0,1451,rbr,.tif,"POLYGON ((-89.46883 52.97854, -89.46841 52.978...",900
1302,105398063,RED60_2012_1899_rbr_.tif,RED60_2012_1899_rbr,RED60_2012_1899,RED60,IFR,2012.0,LTG,MON,2012-07-02,...,2013-03-12,2013-03-12,5345,0,0,1899,rbr,.tif,"POLYGON ((-93.09672 53.37931, -93.09722 53.377...",1302
66,46000261,SLK44_2001_67_rbr_.tif,SLK44_2001_67_rbr,SLK44_2001_67,SLK44,IFR,2001.0,LTG,SUP,2001-07-20,...,2002-09-18,2005-04-28,86,0,0,67,rbr,.tif,"MULTIPOLYGON (((-90.87023 53.85702, -90.87034 ...",66
242,46001566,FOR61_1995_328_rbr_.tif,FOR61_1995_328_rbr,FOR61_1995_328,FOR61,IFR,1995.0,LTG,SUP,1995-06-20,...,2003-07-31,2005-04-28,397,0,0,328,rbr,.tif,"POLYGON ((-91.17922 49.08034, -91.17992 49.080...",242


# Workflow

Take a shapefile and convert to features

In [27]:
# Convert the GeoDataFrame to a list of dictionaries
features = []
for _, row in data.iterrows():
    geom = row['geometry'].__geo_interface__
    properties = row.drop('geometry').to_dict()
    feature = ee.Feature(ee.Geometry(geom), properties)
    features.append(feature)

# Create an Earth Engine FeatureCollection from the list of features
feature_collection = ee.FeatureCollection(features)

In [15]:
# Define the Canadian DEM.
# Define the geometric bounds of Ontario.
ontario_bounds = ee.Geometry.Polygon([
    [-95.156, 41.676],  # Southwest Corner
    [-74.343, 41.676],  # Southeast Corner
    [-74.343, 56.859],  # Northeast Corner
    [-95.156, 56.859],  # Northwest Corner
    [-95.156, 41.676]   # Closing the polygon
])

# Define the Canadian DEM image collection.
canadian_dem_collection = ee.ImageCollection('NRCan/CDEM')

# Mosaic the image collection to create a single image.
mosaiced_dem = canadian_dem_collection.mosaic()

# Clip the mosaiced image by Ontario bounds.
ontario_dem = mosaiced_dem.clip(ontario_bounds)

In [17]:
# Function to calculate TRI for a given DEM array.
def calculate_tri(dem):
    # Check if the DEM array is 2D.
    if dem.ndim != 2:
        raise ValueError("DEM array must be 2D")
    
    rows, cols = dem.shape
    tri = np.zeros((rows, cols))
    offsets = [(-1, -1), (-1, 0), (-1, 1), (0, -1), (0, 1), (1, -1), (1, 0), (1, 1)]
    
    for i in range(1, rows - 1):
        for j in range(1, cols - 1):
            elevation_diffs = []
            for offset in offsets:
                neighbor_i = i + offset[0]
                neighbor_j = j + offset[1]
                elevation_diff = dem[i, j] - dem[neighbor_i, neighbor_j]
                elevation_diffs.append(elevation_diff ** 2)
            tri[i, j] = np.sqrt(np.mean(elevation_diffs))
    
    return tri

In [41]:
# Function to calculate Average Elevation for a given DEM array.
def calculate_average_elevation(dem):
    return np.mean(dem)

# Function to calculate Slope for a given DEM array.
def calculate_slope(dem):
    rows, cols = dem.shape
    slope = np.zeros((rows, cols))
    for i in range(1, rows - 1):
        for j in range(1, cols - 1):
            dzdx = (dem[i+1, j] - dem[i-1, j]) / 2
            dzdy = (dem[i, j+1] - dem[i, j-1]) / 2
            slope[i, j] = np.sqrt(dzdx**2 + dzdy**2)
    return np.mean(slope)

# Function to calculate Terrain Ruggedness Index (TRI) for a given DEM array.
def calculate_tri(dem):
    rows, cols = dem.shape
    tri = np.zeros((rows, cols))
    offsets = [(-1, -1), (-1, 0), (-1, 1), (0, -1), (0, 1), (1, -1), (1, 0), (1, 1)]
    for i in range(1, rows - 1):
        for j in range(1, cols - 1):
            elevation_diffs = []
            for offset in offsets:
                neighbor_i = i + offset[0]
                neighbor_j = j + offset[1]
                elevation_diff = dem[i, j] - dem[neighbor_i, neighbor_j]
                elevation_diffs.append(elevation_diff ** 2)
            tri[i, j] = np.sqrt(np.mean(elevation_diffs))
    return np.mean(tri)

# Function to calculate Topographic Position Index (TPI) for a given DEM array.
def calculate_tpi(dem):
    rows, cols = dem.shape
    tpi = np.zeros((rows, cols))
    offsets = [(-1, -1), (-1, 0), (-1, 1), (0, -1), (0, 1), (1, -1), (1, 0), (1, 1)]
    for i in range(1, rows - 1):
        for j in range(1, cols - 1):
            neighbor_elevations = []
            for offset in offsets:
               (dem[neighbor_i, neighbor_j])
            tpi[i, j] = dem[i, j] - np.mean(neighbor_elevations)
    return np.mean(tpi)

# Function to calculate Standard Deviation of Elevation for a given DEM array.
def calculate_std_elevation(dem):
    return np.std(dem)

# Function to calculate Aspect for a given DEM array.
def calculate_aspect(dem):
    rows, cols = dem.shape
    aspect = np.zeros((rows, cols))
    for i in range(1, rows - 1):
        for j in range(1, cols - 1):
            (dem[i+1, j] - dem[i-1, j]) / 2
            dzdy = (dem[i, j+1] - dem[i, j-1]) / 2
            aspect[i, j] = np.arctan2(dzdy, dzdx) * 180 / np.pi
    return np.mean(aspect)


In [21]:
# Function to get DEM for a fire feature, clip it, and return it as a 2D numpy array.
def get_dem_array(feature):
    # Get the geometry of the feature.
    geometry = feature.geometry()
    
    # Clip the DEM to the feature's geometry.
    dem_clipped = ontario_dem.clip(geometry)
    
    # Define the scale (resolution) for the DEM.
    scale = 30
    
    # Get the DEM data as a numpy array.
    dem_array = geemap.ee_to_numpy(dem_clipped, region=geometry, scale=scale)
    
    # Ensure the DEM array is 2D.
    if dem_array.ndim == 3:
        dem_array = dem_array[:, :, 0]  # Extract the first band if it's a 3D array.
    
    return dem_array

In [33]:
def calculate_statistics(tri_arrays):
    statistics = []
    for tri_data in tri_arrays:
        fire_id = tri_data['Fire_ID']
        tri_array = tri_data['TRI']
        median_tri = np.median(tri_array)
        mean_tri = np.mean(tri_array)
        statistics.append({'Fire_ID': fire_id, 'median_tri': median_tri, 'mean_tri': mean_tri})
    return statistics

In [37]:
    
# Function to process the GeoDataFrame in chunks and calculate TRI.
def get_topo(data):
    tri_arrays = []

    # Loop through the data frame 10 rows at a time.
    for i in range(0, len(data), 10):
        features = []  # Reset the features list for each chunk.
        data_subset = data.iloc[i:i+10]
        
        # Convert the subset to a list of features.
        for _, row in data_subset.iterrows():
            geom = row['geometry'].__geo_interface__
            properties = row.drop('geometry').to_dict()
            feature = ee.Feature(ee.Geometry(geom), properties)
            features.append(feature)

        # Create an Earth Engine FeatureCollection from the list of features.
        feature_collection = ee.FeatureCollection(features)

        # Process each feature in the feature collection.
        features_list = feature_collection.toList(feature_collection.size())
        for j in range(features_list.size().getInfo()):
            feature = ee.Feature(features_list.get(j))
            dem_array = get_dem_array(feature)
            
            # Check if the DEM array is valid before calculating TRI.
            if dem_array is not None and dem_array.size > 0:
                try:
                    tri_array = calculate_tri(dem_array)
                    fire_id = feature.get('Fire_ID').getInfo()
                    tri_arrays.append({'Fire_ID': fire_id, 'TRI': tri_array})
                except ValueError as e:
                    print(f"Error calculating TRI for feature {j}: {e}")
            else:
                print(f"Invalid DEM array for feature {j}")

    # Calculate statistics for each TRI array.
    statistics = calculate_statistics(tri_arrays)

    # Create a DataFrame from the statistics.
    df = pd.DataFrame(statistics)

    return df



In [39]:
df = get_topo(data)
print(df)

              Fire_ID  median_tri  mean_tri
0        NIP53_2001_1    0.866025  0.850927
1        NIP39_2001_2    0.866025  1.003163
2        NIP52_2001_3    0.000000  0.466530
3        NIP35_2001_5    0.000000  0.197253
4        NIP24_2001_7    0.000000  0.265583
5        SLK5_2002_12    0.612372  0.671424
6        SLK6_2002_13    0.790569  0.870327
7        SLK7_2002_14    0.612372  0.660107
8        RED1_2002_15    0.790569  0.854683
9        SLK1_2002_16    0.707107  0.778901
10       SLK4_2002_17    0.612372  0.576870
11       SLK8_2002_18    0.612372  0.632259
12      SLK13_2000_20    0.612372  0.465366
13      NIP26_2000_21    0.000000  0.244841
14      NIP19_2000_22    0.707107  0.749113
15      NIP24_2000_23    0.353553  0.358417
16      SLK18_2000_24    0.612372  0.564583
17      RED12_2000_25    0.935414  1.159964
18      NIP25_2000_26    0.612372  0.531708
19       SLK6_2000_27    0.353553  0.372323
20      SLK16_2000_28    0.353553  0.367630
21      DRY14_2000_30    1.29881

In [40]:
df.to_csv("../data/outputs/on/on_co-occurrences_topo.csv", index=False)